In [ ]:
#scikit learn ensembe workflow for binary probability
import time; start_time = time.time()
import numpy as np
import pandas as pd
from sklearn import ensemble
import xgboost as xgb
from sklearn.metrics import log_loss, make_scorer
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import train_test_split
import random; random.seed(2016)
from sklearn.base import BaseEstimator
from scipy.optimize import minimize

DATA_DIR = "/Users/patrickkennedy/Desktop/Data_Science_MISC/Kaggle"
TRAIN_PATH = "/BNP_Paribas/train.csv"
TEST_PATH = "/BNP_Paribas/test.csv"

train, test, target = load_and_process(DATA_DIR, TRAIN_PATH, TEST_PATH)

#g={'ne':150,'md':6,'mf':80,'rs':2016} #change to g={'ne':500,'md':40,'mf':60,'rs':2016}
g={'ne':500,'md':40,'mf':60,'rs':2016}
etc = ensemble.ExtraTreesClassifier(n_estimators=g['ne'], max_depth=g['md'], max_features=g['mf'], random_state=g['rs'], criterion='entropy', min_samples_split= 4, min_samples_leaf= 2, verbose = 0, n_jobs =-1)      
etr = ensemble.ExtraTreesRegressor(n_estimators=g['ne'], max_depth=g['md'], max_features=g['mf'], random_state=g['rs'], min_samples_split= 4, min_samples_leaf= 2, verbose = 0, n_jobs =-1)      
rfc = ensemble.RandomForestClassifier(n_estimators=g['ne'], max_depth=g['md'], max_features=g['mf'], random_state=g['rs'], criterion='entropy', min_samples_split= 4, min_samples_leaf= 2, verbose = 0, n_jobs =-1)
rfr = ensemble.RandomForestRegressor(n_estimators=g['ne'], max_depth=g['md'], max_features=g['mf'], random_state=g['rs'], min_samples_split= 4, min_samples_leaf= 2, verbose = 0, n_jobs =-1)
xgr = xgb.XGBRegressor(n_estimators=g['ne'], max_depth=g['md'], seed=g['rs'], missing=np.nan, learning_rate=0.02, subsample=0.9, colsample_bytree=0.85, objective='reg:linear')
xgc = xgb.XGBClassifier(n_estimators=g['ne'], max_depth=g['md'], seed=g['rs'], missing=np.nan, learning_rate=0.02, subsample=0.9, colsample_bytree=0.85, objective='binary:logistic') #try 'binary:logitraw'
#clf = {'etc':etc, 'etr':etr, 'rfc':rfc, 'rfr':rfr, 'xgr':xgr, 'xgc':xgc} # use this line instead
clf = {'etc':etc, 'xgc':xgc} # removed due to kaggle performance, would prefer less time and more cores than more time and less cores :)


p_valid, p_test, p_valid_real, p_test_real = run_classifiers(train, target, test, clf, 2)
      

#what about the flog_loss params? uncomment?
#can i condense more?

('Ensemble Model: ', 'etc', ' Best CV score: ', -0.47004251022039067, ' Time: ', 6.71)
('Ensemble Model: ', 'etc', ' Time: ', 9.23)
('Ensemble Model: ', 'etc', ' Time: ', 17.42)
('Ensemble Model: ', 'etc', ' Time: ', 22.02)
('Ensemble Model: ', 'etc', ' Best CV score: ', -0.46794681173707342, ' Time: ', 26.71)
('Ensemble Model: ', 'etc', ' Time: ', 29.25)


In [33]:
preds = optimizer(p_valid, p_test, y_valid, p_valid_real, p_test_real, y_valid_real, y_test, for_real)
pd.DataFrame({"ID": id_test, "PredictedProb": preds[:,1]}).to_csv('owls_script_better_params.csv',index=False)

Performance of optimization based ensemblers (2nd layer) on X_test
------------------------------------------------------------
EN_optA:             logloss  => 0.4641693
Calibrated_EN_optA:  logloss  => 0.4648965
EN_optB:             logloss  => 0.4633164
Calibrated_EN_optB:  logloss  => 0.4751522

REAL: Performance of optimization based ensemblers (2nd layer) on X_test
------------------------------------------------------------


3rd_layer:           logloss  => 0.4642404
3rd_layer:           logloss  => 0.4629070
3rd_layer:           logloss  => 0.4628577
3rd_layer:           logloss  => 0.4628372
3rd_layer:           logloss  => 0.4628219
3rd_layer:           logloss  => 0.4628186
3rd_layer:           logloss  => 0.4628052
3rd_layer:           logloss  => 0.4627997
3rd_layer:           logloss  => 0.4627934


In [27]:
#3rd layer gives .462, kaggle score is .464

In [2]:
def objf_ens_optA(w, Xs, y, n_class):
    """
    Function to be minimized in the EN_optA ensembler.
    
    Parameters:
    ----------
    w: array-like, shape=(n_preds)
       Candidate solution to the optimization problem (vector of weights).
    Xs: list of predictions to combine
       Each prediction is the solution of an individual classifier and has a
       shape=(n_samples, n_classes).
    y: array-like sahpe=(n_samples,)
       Class labels
    n_class: int
       Number of classes in the problem (12 in Airbnb competition)
    
    Return:
    ------
    score: Score of the candidate solution.
    """
    w = np.abs(w)
    sol = np.zeros(Xs[0].shape)
    for i in range(len(w)):
        sol += Xs[i] * w[i]
    #Using log-loss as objective function (different objective functions can be used here). 
    score = log_loss(y, sol)   
    return score
        

class EN_optA(BaseEstimator):
    """
    Given a set of predictions $X_1, X_2, ..., X_n$,  it computes the optimal set of weights
    $w_1, w_2, ..., w_n$; such that minimizes $log\_loss(y_T, y_E)$, 
    where $y_E = X_1*w_1 + X_2*w_2 +...+ X_n*w_n$ and $y_T$ is the true solution.
    """
    def __init__(self, n_class):
        super(EN_optA, self).__init__()
        self.n_class = n_class
        
    def fit(self, X, y):
        """
        Learn the optimal weights by solving an optimization problem.
        
        Parameters:
        ----------
        Xs: list of predictions to be ensembled
           Each prediction is the solution of an individual classifier and has 
           shape=(n_samples, n_classes).
        y: array-like
           Class labels
        """
        #print X.shape[1], self.n_class
        
        Xs = np.hsplit(X, X.shape[1]/self.n_class)
        #Initial solution has equal weight for all individual predictions.
        x0 = np.ones(len(Xs)) / float(len(Xs)) 
        #Weights must be bounded in [0, 1]
        bounds = [(0,1)]*len(x0)   
        #All weights must sum to 1
        cons = ({'type':'eq','fun':lambda w: 1-sum(w)})
        #Calling the solver
        res = minimize(objf_ens_optA, x0, args=(Xs, y, self.n_class), 
                       method='SLSQP', 
                       bounds=bounds,
                       constraints=cons
                       )
        self.w = res.x
        return self
    
    def predict_proba(self, X):
        """
        Use the weights learned in training to predict class probabilities.
        
        Parameters:
        ----------
        Xs: list of predictions to be blended.
            Each prediction is the solution of an individual classifier and has 
            shape=(n_samples, n_classes).
            
        Return:
        ------
        y_pred: array_like, shape=(n_samples, n_class)
                The blended prediction.
        """
        Xs = np.hsplit(X, X.shape[1]/self.n_class)
        y_pred = np.zeros(Xs[0].shape)
        for i in range(len(self.w)):
            y_pred += Xs[i] * self.w[i] 
        return y_pred

In [3]:
def objf_ens_optB(w, Xs, y, n_class):
    """
    Function to be minimized in the EN_optB ensembler.
    
    Parameters:
    ----------
    w: array-like, shape=(n_preds)
       Candidate solution to the optimization problem (vector of weights).
    Xs: list of predictions to combine
       Each prediction is the solution of an individual classifier and has a
       shape=(n_samples, n_classes).
    y: array-like sahpe=(n_samples,)
       Class labels
    n_class: int
       Number of classes in the problem, i.e. = 12
    
    Return:
    ------
    score: Score of the candidate solution.
    """
    #Constraining the weights for each class to sum up to 1.
    #This constraint can be defined in the scipy.minimize function, but doing 
    #it here gives more flexibility to the scipy.minimize function 
    #(e.g. more solvers are allowed).
    w_range = np.arange(len(w))%n_class 
    for i in range(n_class): 
        w[w_range==i] = w[w_range==i] / np.sum(w[w_range==i])
        
    sol = np.zeros(Xs[0].shape)
    for i in range(len(w)):
        sol[:, i % n_class] += Xs[int(i / n_class)][:, i % n_class] * w[i] 
        
    #Using log-loss as objective function (different objective functions can be used here). 
    score = log_loss(y, sol)   
    return score
    

class EN_optB(BaseEstimator):
    """
    Given a set of predictions $X_1, X_2, ..., X_n$, where each $X_i$ has
    $m=12$ clases, i.e. $X_i = X_{i1}, X_{i2},...,X_{im}$. The algorithm finds the optimal 
    set of weights $w_{11}, w_{12}, ..., w_{nm}$; such that minimizes 
    $log\_loss(y_T, y_E)$, where $y_E = X_{11}*w_{11} +... + X_{21}*w_{21} + ... 
    + X_{nm}*w_{nm}$ and and $y_T$ is the true solution.
    """
    def __init__(self, n_class):
        super(EN_optB, self).__init__()
        self.n_class = n_class
        
    def fit(self, X, y):
        """
        Learn the optimal weights by solving an optimization problem.
        
        Parameters:
        ----------
        Xs: list of predictions to be ensembled
           Each prediction is the solution of an individual classifier and has 
           shape=(n_samples, n_classes).
        y: array-like
           Class labels
        """
        #print X.shape[1], self.n_class
        
        Xs = np.hsplit(X, X.shape[1]/self.n_class)
        #Initial solution has equal weight for all individual predictions.
        x0 = np.ones(self.n_class * len(Xs)) / float(len(Xs)) 
        #Weights must be bounded in [0, 1]
        bounds = [(0,1)]*len(x0)   
        #Calling the solver (constraints are directly defined in the objective
        #function)
        res = minimize(objf_ens_optB, x0, args=(Xs, y, self.n_class), 
                       method='L-BFGS-B', 
                       bounds=bounds, 
                       )
        self.w = res.x
        return self
    
    def predict_proba(self, X):
        """
        Use the weights learned in training to predict class probabilities.
        
        Parameters:
        ----------
        Xs: list of predictions to be ensembled
            Each prediction is the solution of an individual classifier and has 
            shape=(n_samples, n_classes).
            
        Return:
        ------
        y_pred: array_like, shape=(n_samples, n_class)
                The ensembled prediction.
        """
        
        Xs = np.hsplit(X, X.shape[1]/self.n_class)
        y_pred = np.zeros(Xs[0].shape)
        for i in range(len(self.w)):
            y_pred[:, i % self.n_class] += \
                   Xs[int(i / self.n_class)][:, i % self.n_class] * self.w[i]  
        return y_pred

In [4]:
def optimizer(p_valid, p_test, y_valid, p_valid_real, p_test_real, y_valid_real, y_test, for_real):
    from sklearn.calibration import CalibratedClassifierCV

    print('Performance of optimization based ensemblers (2nd layer) on X_test')   
    print('------------------------------------------------------------')
    
    #Creating the data for the 2nd layer.
    XV = np.hstack(p_valid)
    XT = np.hstack(p_test)  

    #XV = np.array(p_valid)
    #XT = np.array(p_test)

    n_classes = 2

    #EN_optA
    enA = EN_optA(n_classes)
    enA.fit(XV, y_valid)
    w_enA = enA.w
    y_enA = enA.predict_proba(XT)
    print('{:20s} {:2s} {:1.7f}'.format('EN_optA:', 'logloss  =>', log_loss(y_test, y_enA)))
    
    #Calibrated version of EN_optA 
    cc_optA = CalibratedClassifierCV(enA, method='isotonic')
    cc_optA.fit(XV, y_valid)
    y_ccA = cc_optA.predict_proba(XT)
    print('{:20s} {:2s} {:1.7f}'.format('Calibrated_EN_optA:', 'logloss  =>', log_loss(y_test, y_ccA)))
        
    #EN_optB
    enB = EN_optB(n_classes) 
    enB.fit(XV, y_valid)
    w_enB = enB.w
    y_enB = enB.predict_proba(XT)
    print('{:20s} {:2s} {:1.7f}'.format('EN_optB:', 'logloss  =>', log_loss(y_test, y_enB)))

    #Calibrated version of EN_optB
    cc_optB = CalibratedClassifierCV(enB, method='isotonic')
    cc_optB.fit(XV, y_valid)
    y_ccB = cc_optB.predict_proba(XT)  
    print('{:20s} {:2s} {:1.7f}'.format('Calibrated_EN_optB:', 'logloss  =>', log_loss(y_test, y_ccB)))
    print('')


    if for_real:
        print('REAL: Performance of optimization based ensemblers (2nd layer) on X_test')   
        print('------------------------------------------------------------')
    
        #Creating the data for the 2nd layer.
        XV_real = np.hstack(p_valid_real)
        XT_real = np.hstack(p_test_real)  

        n_classes = 2

        #EN_optA
        enA_real = EN_optA(n_classes)
        enA_real.fit(XV_real, y_valid_real)
        w_enA_real = enA_real.w
        y_enA_real = enA_real.predict_proba(XT_real)
        #print('{:20s} {:2s} {:1.7f}'.format('EN_optA:', 'logloss  =>', log_loss(y_test, y_enA)))
    
        #Calibrated version of EN_optA 
        cc_optA_real = CalibratedClassifierCV(enA_real, method='isotonic')
        cc_optA_real.fit(XV_real, y_valid_real)
        y_ccA_real = cc_optA_real.predict_proba(XT_real)
        #print('{:20s} {:2s} {:1.7f}'.format('Calibrated_EN_optA:', 'logloss  =>', log_loss(y_test, y_ccA)))
        
        #EN_optB
        enB_real = EN_optB(n_classes) 
        enB_real.fit(XV_real, y_valid_real)
        w_enB_real = enB_real.w
        y_enB_real = enB_real.predict_proba(XT_real)
        #print('{:20s} {:2s} {:1.7f}'.format('EN_optB:', 'logloss  =>', log_loss(y_test, y_enB)))

        #Calibrated version of EN_optB
        cc_optB_real = CalibratedClassifierCV(enB_real, method='isotonic')
        cc_optB_real.fit(XV_real, y_valid_real)
        y_ccB_real = cc_optB_real.predict_proba(XT_real)  
        #print('{:20s} {:2s} {:1.7f}'.format('Calibrated_EN_optB:', 'logloss  =>', log_loss(y_test, y_ccB)))
    print('')
    print('')
        
        
    #optimize weighting of the 3rd level - keep the same weighting for real data
    best_score = 10.0

    for i in range(10000):
        first = random.randint(0,20)
        second = random.randint(0,20)
        third = random.randint(0,20)
        fourth = random.randint(0,20)
        total = first + second + third + fourth
        first = first / (total * 1.0)
        second = second / (total * 1.0)
        third = third / (total * 1.0)
        fourth = fourth / (total * 1.0)
    
        y_3l = (y_enA * first) + (y_ccA * second) + (y_enB * third) + (y_ccB * fourth)
        current_score = log_loss(y_test, y_3l)
    
        if current_score < best_score:
            print('{:20s} {:2s} {:1.7f}'.format('3rd_layer:', 'logloss  =>', log_loss(y_test, y_3l)))
            #print first, second, third, fourth
            best_score = current_score
            best_first = first
            best_second = second
            best_third = third
            best_fourth = fourth
            
    preds = (y_enA_real * first) + (y_ccA_real * second) + (y_enB_real * third) + (y_ccB_real * fourth)
    
    return preds

In [9]:
def run_classifiers(train, target, test, clf, jitters):
    y_pred=[]
    p_valid = []
    p_test = []

    p_valid_real = []
    p_test_real = []
    
    for c in clf:
        holder = []
        for i in xrange(jitters):
            dummy = random.randint(1,10000)
            x = True
            while x == True:
                if dummy in holder:
                    dummy = random.randint(1,10000)
                else:
                    x = False
            holder.append(dummy)
            random.seed(dummy)
            
            
            X, X_test, y, y_test = train_test_split(train, target, test_size=0.33, random_state=dummy) #Spliting data into train and test sets.
            X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.33, random_state=dummy) #Spliting train data into training and validation sets.

            X_real = train
            X_test_real = test
            y_real = target
            X_train_real, X_valid_real, y_train_real, y_valid_real = train_test_split(X_real, y_real, test_size=0.33, random_state=dummy)
            LL  = make_scorer(flog_loss, greater_is_better=False)
            
            if c[:1] != "x": #not xgb
                model = GridSearchCV(estimator=clf[c], param_grid={}, n_jobs =-1, cv=2, verbose=0, scoring=LL, random_state=dummy)
                model.fit(X, y)
                best_score = (log_loss(y_test, model.predict_proba(X_test)))*-1
                p_test.append(model.predict_proba(X_test))
                print("Ensemble Model: ", c, " Best CV score: ", best_score, " Time: ", round(((time.time() - start_time)/60),2))
    
                model.fit(X_train, y_train)
                p_valid.append(model.predict_proba(X_valid))
                print("Ensemble Model: ", c, " Time: ", round(((time.time() - start_time)/60),2))
        
                model.fit(X_real, y_real)
                p_test_real.append(model.predict_proba(X_test_real))
                print("Ensemble Model: ", c, " Time: ", round(((time.time() - start_time)/60),2))
    
                model.fit(X_train_real, y_train_real)
                p_valid_real.append(model.predict_proba(X_valid_real))
                print("Ensemble Model: ", c, " Time: ", round(((time.time() - start_time)/60),2))
        
            else: #xgb
                X_fit, X_eval, y_fit, y_eval= train_test_split(X, y, test_size=0.35, train_size=0.65, random_state=dummy)
                model = clf[c]
                model.fit(X_fit, y_fit.values, early_stopping_rounds=20, eval_metric="logloss", eval_set=[(X_eval, y_eval)], verbose=0)
                best_score = (log_loss(y_test, model.predict_proba(X_test)))*-1
                p_test.append(model.predict_proba(X_test))
                print("Ensemble Model: ", c, " Best CV score: ", best_score, " Time: ", round(((time.time() - start_time)/60),2))

                X_fit, X_eval, y_fit, y_eval= train_test_split(X_train, y_train, test_size=0.35, train_size=0.65, random_state=g['rs'])
                model = clf[c]
                model.fit(X_fit, y_fit.values, early_stopping_rounds=20, eval_metric="logloss", eval_set=[(X_eval, y_eval)], verbose=0)
                p_valid.append(model.predict_proba(X_valid))
                print("Ensemble Model: ", c, " Time: ", round(((time.time() - start_time)/60),2))
        
                X_fit, X_eval, y_fit, y_eval= train_test_split(X_real, y_real, test_size=0.35, train_size=0.65, random_state=g['rs'])
                model = clf[c]
                model.fit(X_fit, y_fit.values, early_stopping_rounds=20, eval_metric="logloss", eval_set=[(X_eval, y_eval)], verbose=0)
                p_test_real.append(model.predict_proba(X_test_real))
                print("Ensemble Model: ", c, " Time: ", round(((time.time() - start_time)/60),2))        
    
                X_fit, X_eval, y_fit, y_eval= train_test_split(X_train_real, y_train_real, test_size=0.35, train_size=0.65, random_state=g['rs'])
                model = clf[c]
                model.fit(X_fit, y_fit.values, early_stopping_rounds=20, eval_metric="logloss", eval_set=[(X_eval, y_eval)], verbose=0)
                p_valid_real.append(model.predict_proba(X_valid_real))
                print("Ensemble Model: ", c, " Time: ", round(((time.time() - start_time)/60),2))    
    
    return p_valid, p_test, p_valid_real, p_test_real
    

In [6]:
def load_and_process(DATA_DIR, TRAIN_PATH, TEST_PATH):

    train = pd.read_csv(DATA_DIR + TRAIN_PATH)
    test = pd.read_csv(DATA_DIR + TEST_PATH)

    target = train['target']
    train = train.drop(['target'],axis=1)
    id_test = test['ID']
    num_train = train.shape[0]

    def fill_nan_null(val):
        ret_fill_nan_null = 0.0
        if val == True:
            ret_fill_nan_null = 1.0
        return ret_fill_nan_null

    df_all = pd.concat((train, test), axis=0, ignore_index=True)
    df_all['null_count'] = df_all.isnull().sum(axis=1).tolist()
    df_all_temp = df_all['ID']
    df_all = df_all.drop(['ID'],axis=1)
    df_data_types = df_all.dtypes[:] #{'object':0,'int64':0,'float64':0,'datetime64':0}
    d_col_drops = []

    for i in range(len(df_data_types)):
        df_all[str(df_data_types.index[i])+'_nan_'] = df_all[str(df_data_types.index[i])].map(lambda x:fill_nan_null(pd.isnull(x)))
    df_all = df_all.fillna(-9999)
    #df_all = df_all.replace(0, -9999)

    for i in range(len(df_data_types)):
        if str(df_data_types[i])=='object':
            df_u = pd.unique(df_all[str(df_data_types.index[i])].ravel())
            #print("Column: ", str(df_data_types.index[i]), " Length: ", len(df_u))
            d={}
            j = 1000
            for s in df_u:
                d[str(s)]=j
                j+=5
            df_all[str(df_data_types.index[i])+'_vect_'] = df_all[str(df_data_types.index[i])].map(lambda x:d[str(x)])
            d_col_drops.append(str(df_data_types.index[i]))
            if len(df_u)<150:
                dummies = pd.get_dummies(df_all[str(df_data_types.index[i])]).rename(columns=lambda x: str(df_data_types.index[i]) + '_' + str(x))
                df_all_temp = pd.concat([df_all_temp, dummies], axis=1)

    df_all_temp = df_all_temp.drop(['ID'],axis=1)
    df_all = pd.concat([df_all, df_all_temp], axis=1)
    #print(len(df_all), len(df_all.columns))
    #df_all.to_csv("df_all.csv")
    train = df_all.iloc[:num_train]
    test = df_all.iloc[num_train:]
    train = train.drop(d_col_drops,axis=1)
    test = test.drop(d_col_drops,axis=1)
    
    return train, test, target

In [7]:
def flog_loss(ground_truth, predictions):
    flog_loss_ = log_loss(ground_truth, predictions) #, eps=1e-15, normalize=True, sample_weight=None)
    return flog_loss_